# Transform and clean data from World Bank

### data from the ZIP file source : https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators

1) extract data from sheet : data
2) extract countries  from sheet : Countries (need to get the region from this sheet)
3) extract indicators : filter of needed indicators and their assignment to categories

4) unpivot the data and use years as long
5) drop NAN values
6) merge data with countries to get the region
6) drop NAN values representaing regions aggregation
7) convert 'years to int
8) filter the data to get statistics from 2000

9) merge with indicators_df to extract only needed indicators and their category
10) generate a csv file for each category of indicators

In [1]:
import pandas as pd

In [4]:
# import data

path = '../Resources_external/WDI_EXCEL_2025_01_28.xlsx'
sheet  = 'Data'
data_df = pd.read_excel(path, sheet_name=sheet )

data_df.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.488497,18.001597,18.558234,19.043572,19.586457,20.192064,20.828814,21.372164,22.100884,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.811504,7.096003,7.406706,7.666648,8.020952,8.403358,8.718306,9.097176,9.473374,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,38.152090,38.488233,38.779953,39.068462,39.445526,39.818645,40.276374,40.687817,41.211606,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.871956,33.922276,38.859598,40.223744,43.035073,44.390861,46.282371,48.127211,48.742043,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.672943,16.527554,24.627753,25.432092,27.061929,29.154282,31.022083,32.809138,33.760782,NaN


In [5]:
# import Countries (we need region to aggregate)

path = '../Resources_Output/countries_UN_referential.csv'

Country_df = pd.read_csv(path)

Country_df.head(5)

,iso3Code,iso2Code,country,region,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,Oranjestad,-70.0167,12.51670
1,AFG,AF,Afghanistan,South Asia,Kabul,69.1761,34.52280
2,AGO,AO,Angola,Sub-Saharan Africa,Luanda,13.2420,-8.81155
3,ALB,AL,Albania,Europe & Central Asia,Tirane,19.8172,41.33170
4,AND,AD,Andorra,Europe & Central Asia,Andorra la Vella,1.5218,42.50750


In [6]:
# import indicators selection

path = "../Resources_external/indicators_selection.xlsx"
indicators_df = pd.read_excel(path )

## filter the selected indicators
x = (indicators_df['select'] == 'y')
indicators_df = indicators_df[x]
## drop column 'select'
indicators_df = indicators_df.drop(columns = {'select'})
indicators_df.head(5)

,Indicator Name,Indicator Code,Category
0,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,energy
3,Access to electricity (% of population),EG.ELC.ACCS.ZS,energy
14,Account ownership at a financial institution o...,FX.OWN.TOTL.YG.ZS,population
51,Adolescents out of school (% of lower secondar...,SE.SEC.UNER.LO.ZS,education
52,"Adolescents out of school, female (% of female...",SE.SEC.UNER.LO.FE.ZS,education


In [7]:
data_df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023'],
      dtype='object')

In [8]:
shape_original = data_df.shape

# unpivot and get the years as a column

data_lg_df = pd.melt(
                    data_df, 
                    id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',],
                    value_vars=['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
                                '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
                                '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
                                '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
                                '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
                                '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
                                '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
                                '2023'],
                     var_name='years' ,           
                     value_name='Value' )

shape_unpivot = data_lg_df.shape

# drop NAN values

data_lg_df  = data_lg_df.dropna(how='any')

shape_cleanna = data_lg_df.shape

data_lg_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,years,Value
50,Africa Eastern and Southern,AFE,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,135.793291
56,Africa Eastern and Southern,AFE,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,88.967697
57,Africa Eastern and Southern,AFE,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,5.631542
58,Africa Eastern and Southern,AFE,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,82.969998
84,Africa Eastern and Southern,AFE,Aquaculture production (metric tons),ER.FSH.AQUA.MT,1960,380.000000


In [9]:
print(f'shape_oringinal : {shape_original}')
print(f'shape_unpivot : {shape_unpivot}')
print(f'shape_cleanna : {shape_cleanna}')

shape_oringinal : (397936, 68)
shape_unpivot : (25467904, 6)
shape_cleanna : (8888933, 6)


In [17]:
# merge data_df with countries-df

data_full_df = pd.merge(data_lg_df, Country_df, left_on='Country Code', right_on='iso3Code')


In [18]:
data_full_df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       'years', 'Value', 'iso3Code', 'iso2Code', 'country', 'region',
       'capitalCity', 'longitude', 'latitude'],
      dtype='object')

In [20]:
data_full_df = data_full_df[['Country Name', 'Country Code', 'Indicator Name',  "region" , 'Indicator Code',
       'years', 'Value']]

# drop NAN values : these NAN values represent aggregation of regions we dont need them 

data_full_df  = data_full_df.dropna(how='any')
data_full_df.head()

,Country Name,Country Code,Indicator Name,region,Indicator Code,years,Value
0,Afghanistan,AFG,"Adolescent fertility rate (births per 1,000 wo...",South Asia,SP.ADO.TFRT,1960,140.598000
1,Afghanistan,AFG,Age dependency ratio (% of working-age populat...,South Asia,SP.POP.DPND,1960,81.061389
2,Afghanistan,AFG,"Age dependency ratio, old (% of working-age po...",South Asia,SP.POP.DPND.OL,1960,5.112019
3,Afghanistan,AFG,"Age dependency ratio, young (% of working-age ...",South Asia,SP.POP.DPND.YG,1960,75.949370
4,Afghanistan,AFG,Aquaculture production (metric tons),South Asia,ER.FSH.AQUA.MT,1960,0.000000


In [21]:
# filter to get data from 2000

## convert years to int

data_full_df = data_full_df.astype({'years' : 'int32'})

## filter

date = data_full_df['years']>2000
data_full_df = data_full_df[date]

data_full_df.head()

,Country Name,Country Code,Indicator Name,region,Indicator Code,years,Value
3284842,Afghanistan,AFG,Access to clean fuels and technologies for coo...,South Asia,EG.CFT.ACCS.ZS,2001,6.6
3284843,Afghanistan,AFG,Access to clean fuels and technologies for coo...,South Asia,EG.CFT.ACCS.RU.ZS,2001,1.0
3284844,Afghanistan,AFG,Access to clean fuels and technologies for coo...,South Asia,EG.CFT.ACCS.UR.ZS,2001,29.9
3284845,Afghanistan,AFG,Access to electricity (% of population),South Asia,EG.ELC.ACCS.ZS,2001,9.3
3284846,Afghanistan,AFG,"Access to electricity, urban (% of urban popul...",South Asia,EG.ELC.ACCS.UR.ZS,2001,74.8


In [22]:
print(f'data_full_df : {data_full_df.shape}')

data_full_df : (4075302, 7)


In [24]:
#merge with indicators to extract only needed indicators data

data_filtred_df = pd.merge(data_full_df, indicators_df, on='Indicator Code')

data_filtred_df = data_filtred_df.rename(columns={'Indicator Name_x' : 'Indicator Name'})

data_filtred_df = data_filtred_df[['Country Name', 'Country Code','region', 'Indicator Code', 'Indicator Name','Category',
        'years', 'Value'
       ]]

data_filtred_df.head(5)

,Country Name,Country Code,region,Indicator Code,Indicator Name,Category,years,Value
0,Afghanistan,AFG,South Asia,EG.CFT.ACCS.ZS,Access to clean fuels and technologies for coo...,energy,2001,6.600000
1,Afghanistan,AFG,South Asia,EG.ELC.ACCS.ZS,Access to electricity (% of population),energy,2001,9.300000
2,Afghanistan,AFG,South Asia,AG.LND.IRIG.AG.ZS,Agricultural irrigated land (% of total agricu...,agriculture,2001,5.662125
3,Afghanistan,AFG,South Asia,AG.LND.AGRI.ZS,Agricultural land (% of land area),agriculture,2001,57.947350
4,Afghanistan,AFG,South Asia,AG.LND.AGRI.K2,Agricultural land (sq. km),agriculture,2001,377950.000000


In [25]:
print(f'data_filtred_df : {data_filtred_df.shape}')

data_filtred_df : (646701, 8)


In [28]:
# export csv file for each category of indicators

indicators = list(data_filtred_df['Category'].unique())

for i in indicators :
    x = data_filtred_df['Category'] == i
    df = data_filtred_df[x]
    df = df.pivot(
                    index=[ 'Country Name',	'Country Code',	'region', 'years'], 
                    columns='Indicator Name', 
                    values='Value'
                    )
    path_output = f'../Outputs/csv_indicators/world_bank_data_{i}.csv'
    df.to_csv(path_output)

    
